In [10]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [11]:
url = "http://api.openweathermap.org/data/2.5/weather?"

In [12]:
data_frame_city = pd.read_csv("data_frame_city.csv")
data_frame_city

,Unnamed: 0,city,lat,lng,max temp,humidity,cloudiness,wind speed,country,Date
0,0,Ürümqi,43.8010,87.6005,265.68,79,100,0.00,CN,1638641613
1,1,Wuhu,31.3366,118.3755,273.10,57,7,2.63,CN,1638641803
2,2,Port Elizabeth,-33.9180,25.5701,291.31,77,90,6.17,ZA,1638641804
3,3,Olafsvik,64.8945,-23.7142,274.82,78,75,2.76,IS,1638641804
4,4,Avarua,-21.2078,-159.7750,298.18,88,75,1.03,CK,1638641804
...,...,...,...,...,...,...,...,...,...,...
588,588,Vavoua,7.3819,-6.4778,302.86,42,57,1.47,CI,1638641857
589,589,Dudinka,69.4058,86.1778,237.38,100,93,1.36,RU,1638641740
590,590,Nānākuli,21.3906,-158.1547,292.99,80,90,2.57,US,1638641857
591,591,Mkushi,-13.6202,29.3939,296.92,32,12,2.69,ZM,1638641857


In [45]:
gmaps.configure(g_key)
fig = gmaps.figure()
subset = data_frame_city[["lat","lng"]]

tuples = [tuple(x) for x in subset.values]
locations = tuples
fig.add_layer(gmaps.heatmap_layer(locations, weights = data_frame_city["humidity"]))
fig

Figure(layout=FigureLayout(height='420px'))

In [46]:
names = data_frame_city["city"]
symbol_layer = gmaps.marker_layer(locations, hover_text=names)
fig.add_layer(symbol_layer)
fig

Figure(layout=FigureLayout(height='420px'))